In [ ]:
from Models.GlassBox.ebm.ebm_class import EBM_class
from interpret.glassbox import ExplainableBoostingRegressor
import matplotlib.pyplot as  plt
import pandas as pd
%reload_ext autoreload
%autoreload 2

In [ ]:
EBM_parameter = dict(
    learning_rate= [0.005, 0.01, 0.02, 0.03],
    min_samples_leaf=[2, 3, 4, 5, 6, 7, 8, 9, 10],
    max_leaves=[2, 3, 4, 5, 6, 7],
    interactions=[10, 15, 20],
)
gridsearch_parameters = dict(
    path="./../../../outputs/scores/",
    name="0",
    nDCG_at=15
)

# Grid-search

In [ ]:
gs = EBM_class(**gridsearch_parameters)

In [ ]:
best_ = gs.grid_search(
    EBMModel=ExplainableBoostingRegressor,  
    hyperparameters=EBM_parameter)
gs.save_model(best_[0], name="EBMReg")

# Metric evaluation

In [ ]:
best_model = gs.load_model(name="./saved_models/EBMReg")

In [ ]:
best_model   

In [ ]:
# nDCG on test-set
nDCG_train = gs.eval_model(model=best_model, df=gs.train, nDCG_at=[1,10,15])
nDCG_valid = gs.eval_model(model=best_model, df=gs.valid, nDCG_at=[1,10,15])
nDCG_test = gs.eval_model(model=best_model, df=gs.test, nDCG_at=[1,10,15])

display(pd.DataFrame([nDCG_train,nDCG_valid,nDCG_test],index=["Training","Validation","Test"]))

In [ ]:
from interpret import show
show(best_model.explain_global())

# Explanation

In [ ]:
best_model = gs.load_model(name="./saved_models/EBMReg")

In [ ]:
best_model

In [ ]:
f, axs = plt.subplots(ncols=3,nrows=4,figsize=(20,15))

for i, feature in enumerate(best_model.feature_names):

    x,y = gs.explanation(best_model, i)
    axs[int(i/3)][i%3].set_xlabel("Value")
    axs[int(i/3)][i%3].set_ylabel("Contribute")
    axs[int(i/3)][i%3].grid()
    axs[int(i/3)][i%3].plot(x,y, label=feature, color="#a74e25")
    axs[int(i/3)][i%3].legend()
plt.show()